In [ ]:
! pip3 install -qU guardrails-ai openai langchain_community langchain_experimental langchain-upstage sentence-transformers langchainhub langchain-chroma langchain matplotlib python-dotenv tavily-python ragas faiss-cpu tokenizers getpass4
! pip3 install -q arize-phoenix[evals]

In [ ]:
import os
import getpass
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [ ]:
from langchain_upstage import UpstageDocumentParseLoader, UpstageGroundednessCheck, ChatUpstage, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.utils.math import cosine_similarity

from IPython.display import display, HTML
from tokenizers import Tokenizer

In [ ]:
doc_loader = UpstageDocumentParseLoader("sample_data_upsight.pdf", output_format='html', coordinates=False)
docs = doc_loader.load()

In [ ]:
import langchain_core
print(dir(langchain_core))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_api', 'agents', 'beta', 'caches', 'callbacks', 'chat_history', 'document_loaders', 'documents', 'embeddings', 'env', 'example_selectors', 'exceptions', 'globals', 'language_models', 'load', 'messages', 'metadata', 'output_parsers', 'outputs', 'prompt_values', 'prompts', 'rate_limiters', 'retrievers', 'runnables', 'surface_langchain_beta_warnings', 'surface_langchain_deprecation_warnings', 'tools', 'tracers', 'utils', 'vectorstores']


In [ ]:
for doc in docs:
    pprint(doc.page_content[:100])

("<p id='0' data-category='paragraph' style='font-size:14px'>Name, Rate, "
 "Review</p><br><p id='1' data-")


In [ ]:
display(HTML(docs[0].page_content[:500]))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

print("Splits:", len(splits))

Splits: 33


In [ ]:
vectorstore = Chroma.from_documents(
    documents=splits, embedding=UpstageEmbeddings(model="solar-embedding-1-large")
)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

In [ ]:
prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    나는 숙박업소의 사장이야. 나는 나의 숙박업소에 등록된 리뷰를 정리해서 각 세부 분야별 장점과 부족한 점을 찾아 더 나은 방향으로 개선하려고 해. 또한 경쟁업체와의 비교를 통해 우리 업소가 취해야 될 전략을 알고 싶어.
    씨마크 호텔의 장소 및 위치 / 서비스 및 친절도 / 숙소 분위기 및 객실 / 편의성 및 시설 / 가격 대비 가치 등등으로 세부 평가기준을 나누어 100점 만점에 몇 점인지 환산해서 알려줘.
    또한 업소가 위치한 장소, 성수기와 비수기 등 다양한 점을 고려하여 매출을 높일 수 있는 전략을 제시해줘. 아래 채팅에 우리 업소에 유저가 등록한 리뷰를 보내줄게. 이를 참고하여 개선방안을 작성해줘.
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)

In [ ]:
# 4. retrive
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
result_docs = retriever.invoke(query)
print(len(result_docs))
print(result_docs[0].page_content[:100])

3
style='font-size:18px'>강릉씨티호텔, 4.2, 강릉 여행하면서 이용한 숙소입니다. 기계식주차장이라서 불편하긴 한데</p><br><p id='50' data-cat


In [ ]:
query= "나는 나의 숙박업소에 등록된 리뷰를 정리해서 각 세부 분야별 장점과 부족한 점을 찾아 더 나은 방향으로 개선하려고 해. "

In [ ]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"question": query, "Context": result_docs})

'강릉씨티호텔의 리뷰를 보면, 대부분의 고객들이 숙소의 친절함과 청결함을 긍정적으로 평가하고 있습니다. 그러나 주차장이 기계식이라서 불편하다는 의견과 샤워기 헤더 각도 조절이 안 된다는 의견도 있습니다.\n\n장소 및 위치: 100점 만점에 90점\n서비스 및 친절도: 100점 만점에 95점\n숙소 분위기 및 객실: 100점 만점에 88점\n편의성 및 시설: 100점 만점에 86점\n가격 대비 가치: 100점 만점에 92점\n\n매출 증대를 위한 전략으로는, 주차장이 불편하다는 의견을 고려하여 주차장을 개선하는 것이 좋을 것입니다. 또한, 주말 가격이 평일보다 비싸다는 의견이 있으므로, 주말 가격을 조금 낮추는 것도 고려해 볼 수 있습니다.\n\n또한, 강릉씨티호텔은 청결하고 가성비가 좋다는 의견이 많으므로, 이러한 장점을 강조하는 마케팅 전략도 효과적일 것입니다. 예를 들어, "청결한 숙소에서 편안한 휴식을 취하세요"라는 메시지를 전달하거나, "가성비 좋은 숙소에서 즐거운 여행을 즐기세요"라는 메시지를 전달할 수 있습니다. 또한, 주변 맛집이나 관광지와 협력하여 패키지를 제공하는 것도 매출 증대에 도움이 될 수 있습니다.'

In [ ]:
query= "강릉씨티호텔이 위치한 장소, 성수기와 비수기 등 다양한 점을 고려하여 매출을 높일 수 있는 전략을 제시해줘. "

In [ ]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"question": query, "Context": result_docs})

'강릉씨티호텔의 장소 및 위치:\n고객들의 리뷰를 보면 호텔의 위치에 대해 언급하는 경우가 많습니다. 일부 고객은 호텔의 위치를 좋다고 평가하고, 다른 고객은 주차장이 불편하다고 언급합니다.\n\n강릉씨티호텔의 서비스 및 친절도:\n대부분의 고객은 강릉씨티호텔의 직원들이 친절하다고 언급하며 긍정적인 평가를 내립니다.\n\n강릉씨티호텔의 숙소 분위기 및 객실:\n리뷰에 따르면 강릉씨티호텔의 숙소 분위기와 객실은 대체로 긍정적인 평가를 받습니다. 일부 고객은 객실이 깨끗하고 청결하다고 언급하며, 다른 고객은 샤워기 헤드의 각도 조절이 안 된다고 언급합니다.\n\n강릉씨티호텔의 편의 시설 및 시설:\n고객들은 강릉씨티호텔의 편의 시설과 시설에 대해 대체로 긍정적인 평가를 내립니다. 일부 고객은 호텔이 깨끗하고 시설이 좋다고 언급하며, 다른 고객은 주차장이 불편하다고 언급합니다.\n\n강릉씨티호텔의 가격 대비 가치:\n일부 고객은 강릉씨티호텔의 가격 대비 가치가 좋다고 언급하며, 다른 고객은 주말 가격이 비싸다고 언급합니다.\n\n매출 증대를 위한 전략:\n강릉씨티호텔은 다양한 방법으로 매출을 증대할 수 있습니다. 예를 들어, 호텔을 홍보하기 위해 온라인 예약 플랫폼과 협력하거나, 지역 관광지와 제휴하여 패키지를 제공할 수 있습니다. 또한, 주차 문제를 해결하기 위해 주차 시설을 개선하거나, 고객 만족도를 높이기 위해 고객 경험을 개선하는 등의 노력을 기울일 수 있습니다.'

In [ ]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"question": query, "Context": result_docs})

'문맥에서 제시된 리뷰를 보면, 강릉씨티호텔은 전반적으로 좋은 평가를 받고 있지만, 주차장과 샤워기 헤드에 대한 불만이 있습니다. 이를 개선하기 위해, 주차장을 더 편리하게 만들고, 샤워기 헤드를 개선할 수 있는 방법을 고민해보시는 것이 좋을 것 같습니다. 또한, 성수기와 비수기에 따른 가격 정책을 조정하여, 비수기에 더 많은 고객을 유치할 수 있도록 하는 것도 좋은 전략일 것입니다.'

In [ ]:
query= "강릉씨티호텔이 위치한 장소, 성수기와 비수기 등 다양한 점을 고려하여 매출을 높일 수 있는 전략을 제시해줘. "

In [ ]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"question": query, "Context": result_docs})

'숙박업소의 사장은 각 세부 분야별 장점과 부족한 점을 파악하고, 경쟁사와의 비교를 통해 전략을 수립하기 위해 다음과 같은 피드백을 제안할 수 있습니다:\n\n1. 주차: 고객들은 기계식 주차장이 불편하다는 의견이 많았습니다. 이를 개선하기 위해 발렛 파킹 서비스를 제공하거나, 외부 주차장을 마련하는 등의 대안을 고려할 수 있습니다.\n2. 직원 서비스: 고객들은 직원들의 친절함에 대해 긍정적인 평가를 내렸습니다. 이를 유지하기 위해 직원 교육 및 서비스 향상에 집중할 수 있습니다.\n3. 숙소 청결도: 고객들은 숙소의 청결도에 대해 전반적으로 긍정적인 평가를 내렸습니다. 이를 유지하기 위해 청소 및 위생 관리에 신경을 쓸 수 있습니다.\n4. 위치: 고객들은 숙소의 위치에 대해 긍정적인 평가를 내렸습니다. 이를 활용하여 마케팅 전략에 활용할 수 있습니다.\n5. 가격: 고객들은 주말 가격과 가격 대비 시설에 대해 다양한 의견을 내렸습니다. 이를 고려하여 가격 정책을 조정하거나, 시설 개선에 집중할 수 있습니다.\n6. 경쟁사와의 비교: 경쟁사와의 비교를 통해 강점을 부각시키고, 약점을 보완하는 전략을 수립할 수 있습니다. 예를 들어, 가격 대비 시설이 우수한 점을 강조하거나, 경쟁사보다 더 많은 편의 시설을 제공하는 등의 전략을 고려할 수 있습니다.'

In [ ]:
query="강릉씨티호텔의 고객의 반응을 분석하여 피드백을 제안"

In [ ]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"question": query, "Context": result_docs})

'경쟁 업체와 비교하여 강릉 씨티호텔이 개선해야 할 점은 다음과 같습니다.\n\n1. 주차 시설 개선: 리뷰에서는 기계식 주차장이 불편하다는 의견이 있었습니다. 이를 개선하여 고객들의 주차 경험을 향상시킬 수 있습니다.\n2. 샤워기 헤드 각도 조절: 일부 리뷰에서는 샤워기 헤드 각도 조절이 불가능하다는 불편함이 언급되었습니다. 이를 개선하여 고객들의 편의성을 높일 수 있습니다.\n3. 주말 가격 조정: 몇몇 리뷰에서는 주말 가격이 평일보다 비싸다는 의견이 있었습니다. 이를 고려하여 주말 가격을 조정하여 고객들의 경제적 부담을 줄일 수 있습니다.\n\n이러한 개선 사항을 고려하여 강릉 씨티호텔은 고객들의 만족도를 높이고 경쟁력을 강화할 수 있을 것입니다.'

In [ ]:
query = "호텔 탑스텐의 시장성를 예측하여 기업 개선 전략 도출"

In [ ]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"question": query, "Context": result_docs})

'문맥에서 제시된 리뷰를 기반으로 강릉씨티호텔의 세부 평가 기준을 점수로 환산하면 다음과 같습니다:\n\n1. 장소 및 위치: 4.2/5\n2. 서비스 및 친절도: 5.0/5\n3. 숙소 분위기 및 객실: 4.6/5\n4. 편의성 및 시설: 4.6/5\n5. 가성비: 4.4/5\n\n문맥에서 제시된 다양한 요소를 고려하여 강릉씨티호텔의 매출을 높일 수 있는 전략은 다음과 같습니다:\n\n1. 기계식 주차장의 불편함을 해결하기 위해 직원들은 손님들에게 주차에 대한 안내를 제공하고 도움을 줄 수 있습니다.\n2. 손님들의 긍정적인 피드백에 따라 친절하고 청결한 서비스를 유지하여 고객 만족도를 높일 수 있습니다.\n3. 주말 가격의 경쟁력을 유지하면서도 손님의 만족도를 높이기 위해 주말 가격을 조금 더 낮출 수 있는 방안을 고려할 수 있습니다.'